# Crime in Chicago: A Year in Review
**Authors**: Srikar Manikonda, Justin Ward, Vanisha Nagali, Rishabh Shah

This interactive report analyzes a year of reported crimes in Chicago to uncover patterns in criminal activity, law enforcement response, and neighborhood characteristics. The insights here are made public using data visualization for ease of understanding and engagement.

In [1]:
import pandas as pd
import altair as alt

alt.data_transformers.disable_max_rows()

df = pd.read_csv("Crimes_-_One_year_prior_to_present (1).csv")
df['DATE  OF OCCURRENCE'] = pd.to_datetime(df['DATE  OF OCCURRENCE'], errors='coerce')
df = df.dropna(subset=['DATE  OF OCCURRENCE'])
df['Month'] = df['DATE  OF OCCURRENCE'].dt.to_period('M').astype(str)

<ipython-input-1-1adbf4b76f7a>:7: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['DATE  OF OCCURRENCE'] = pd.to_datetime(df['DATE  OF OCCURRENCE'], errors='coerce')


In [2]:
theft_df = df[df[' PRIMARY DESCRIPTION'] == 'THEFT']
monthly_theft = theft_df.groupby('Month').size().reset_index(name='Count')

theft_chart=alt.Chart(monthly_theft).mark_line(point=True).encode(
    x=alt.X('Month:T', title='Month'),
    y=alt.Y('Count:Q', title='Number of Thefts'),
    tooltip=['Month', 'Count']
).properties(
    title='Monthly Theft Reports in Chicago',
    width=700,
    height=400
).interactive()
theft_chart

alt.Chart(...)

In [3]:
top_crimes = df[' PRIMARY DESCRIPTION'].value_counts().nlargest(10).reset_index()
top_crimes.columns = ['Crime Type', 'Count']

top_crimes_chart=alt.Chart(top_crimes).mark_bar().encode(
    x=alt.X('Count:Q'),
    y=alt.Y('Crime Type:N', sort='-x'),
    color='Crime Type:N',
    tooltip=['Crime Type', 'Count']
).properties(
    title='Top 10 Reported Crimes in Chicago',
    width=700,
    height=400
)
top_crimes_chart

alt.Chart(...)

In [4]:
arrest_counts = df.groupby([' PRIMARY DESCRIPTION', 'ARREST']).size().unstack(fill_value=0)

true_col = 'Y'
false_col = 'N'

true_vals = arrest_counts[true_col] if true_col in arrest_counts else 0
false_vals = arrest_counts[false_col] if false_col in arrest_counts else 0

total_counts = true_vals + false_vals
non_zero_index = total_counts[total_counts > 0].index

arrest_counts = arrest_counts.loc[non_zero_index]
arrest_counts['Arrest Rate'] = true_vals.loc[non_zero_index] / total_counts.loc[non_zero_index]

arrest_top = arrest_counts.reset_index().sort_values('Arrest Rate', ascending=False).head(10)

arrest_chart=alt.Chart(arrest_top).mark_bar().encode(
    x=alt.X('Arrest Rate:Q'),
    y=alt.Y(' PRIMARY DESCRIPTION:N', sort='-x', title='Crime Type'),
    tooltip=[' PRIMARY DESCRIPTION', 'Arrest Rate']
).properties(
    title='Top Crime Types by Arrest Rate',
    width=700,
    height=400
)
arrest_chart

alt.Chart(...)

## Making Sense of Crime in Chicago

This public dashboard explores crime in Chicago over the past year using data from the city's open data portal. Our central visualization focuses on theft, the most frequently reported crime. The trendline reveals seasonal variation in theft activity, with peaks during the holiday months and dips in the warmer summer periods. This interactive chart helps users intuitively observe how theft evolves over time.

To add broader context, we also explore the top 10 crime types, revealing that battery, criminal damage, and assault follow closely behind theft. These categories show where police resources are likely most needed. We also dive into arrest rates by crime type — crimes like narcotics and weapons violations have much higher arrest rates than property crimes, which may suggest a focus on proactive enforcement rather than reactive response.

Together, these visualizations provide a clear and accessible view of how crime is distributed, reported, and addressed in Chicago. They serve as tools for both concerned citizens and policymakers looking to make informed, data-driven decisions.

## Sources & Citations

- [Crimes Dataset – City of Chicago](https://data.cityofchicago.org/Public-Safety/Crimes-One-year-prior-to-present/ijzp-q8t2)
- [Analysis Notebook on GitHub](https://github.com/YOUR_USERNAME/YOUR_REPO/blob/main/final_notebook.ipynb)

## Contextual Datasets

1. [ACS 5-Year Data by Ward](https://data.cityofchicago.org/Community-Economic-Development/ACS-5-Year-Data-by-Ward/awnk-6fvc)
2. [Chicago Population Statistics](https://data.cityofchicago.org/Health-Human-Services/Chicago-Population-Counts/85cm-7uqa)
3. [CTA GTFS Transit Feed](https://www.transitchicago.com/downloads/sch_data/google_transit.zip)
4. [Police District Boundaries](https://data.cityofchicago.org/Public-Safety/Boundaries-Police-Districts-current-/fthy-xz3r)
5. [Streetlights All Out Map](https://data.cityofchicago.org/Service-Requests/Map-of-Street-Lights-All-Out/5w22-e46m)

In [5]:
theft_chart.save('theft_chart.html')
top_crimes_chart.save('top_crimes_chart.html')
arrest_chart.save('arrest_chart.html')